In [ ]:
"""
Purpose: To visualize a proofread neuron
in both ipyvolume and neuroglancer

"""

# Setting up Access to meshAfterParty package

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/meshAfterParty/")

In [6]:
import proofread_verification as pv
import datajoint_utils as du

INFO - 2021-09-13 03:01:38,054 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-09-13 03:01:38,060 - settings - Setting database.user to celiib
INFO - 2021-09-13 03:01:38,061 - settings - Setting database.password to newceliipass
INFO - 2021-09-13 03:01:38,062 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-09-13 03:01:38,063 - settings - Setting database.user to celiib
INFO - 2021-09-13 03:01:38,064 - settings - Setting database.password to newceliipass
INFO - 2021-09-13 03:01:38,069 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/de

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-09-13 03:01:38,792 - settings - Setting enable_python_native_blobs to True
INFO - 2021-09-13 03:01:38,863 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-09-13 03:01:38,864 - settings - Setting database.user to celiib
INFO - 2021-09-13 03:01:38,866 - settings - Setting database.password to newceliipass
INFO - 2021-09-13 03:01:38,869 - settings - Setting enable_python_native_blobs to True
INFO - 2021-09-13 03:01:39,647 - settings - Setting enable_python_native_blobs to True
INFO - 2021-09-13 03:01:39,738 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-09-13 03:01:39,740 - settings - Setting database.user to celiib
INFO - 2021-09-13 03:01:39,741 - settings - Setting database.password to newceliipass
INFO - 2021-09-13 03:01:39,742 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-09-13 03:01:39,744 - settings - Setting database.user to celiib
INFO - 2021-09-13 03:01:39,745 - settings - Setting database.pa

# Listing the Proofread Neurons in the Database

In [ ]:
du.proofreading_neurons_table()

# Selecting a Subset of Proofread Neurons

In [ ]:
(du.proofreading_neurons_table()
    & "cell_type = 'excitatory'" 
    & 'axon_skeletal_length > 1000000'
    & 'apical_total_skeletal_length > 1000000')

# Picking the Neuron and Visualizing it

In [ ]:
segment_id = 864691134917416714
split_index = 0

In [ ]:
pv.plot_proofread_neuron(
    segment_id,
    split_index,
                        )

In [ ]:
pv.plot_proofread_neuron(
    segment_id,
    split_index,
    plot_synapses=False,
                        )

In [ ]:
pv.plot_proofread_neuron(
    segment_id,
    split_index,
    plot_synapses=True,
    synapse_plot_type="valid_presyn_postsyn",
    plot_error_synapses=False,
    plot_error_mesh=False,
    
                        )

# how to construct own synapse queries and colors

In [ ]:
"""
Notes on querying synapses: 

- error synapses have soma distance of -1
- soma synapses also have soma distance of -2 (but compartment is also labeled soma)
- head/neck/shaft is the category for the 
spine classification (includes boutons/non_boutons)

{'no_label': -1,
 'head': 0,
 'neck': 1,
 'shaft': 2,
 'no_head': 3,
 'bouton': 4,
 'non_bouton': 5}

"""

In [ ]:
import synapse_utils as syu
synapses_objs = pv.syanpse_objs_from_segment_id(segment_id,
                                                split_index)
spine_relevant_features = ["syn_id",
                           "syn_type",
                           "volume",
                           "soma_distance",
                           "head_neck_shaft",
                           "compartment",]
syn_df = syu.synapses_df(synapses_objs)[spine_relevant_features]
syn_df

In [ ]:
syanpse_queries = [
    "(syn_type=='presyn') and (compartment == 'axon') and (soma_distance < 100000)",
    "(syn_type=='presyn') and (compartment == 'axon') and (soma_distance >= 100000)"
]

syanpse_colors = ["blue","aqua"]

pv.plot_proofread_neuron(segment_id,
                        split_index,
                         plot_error_mesh=False,
                        plot_synapses=True,
                         plot_error_synapses=False,
                        synapse_queries=syanpse_queries,
                        synapse_queries_colors=syanpse_colors,
                        syanses_size = 0.3)

# Multiple Neuron Visualizations

In [ ]:
import allen_utils as alu


print(f"Allen Proofreading Bounding Box Corners:")
print(alu.allen_column_bounding_box_corners)
print(f"Allen Proofreading Bbox Center:")
print(alu.allen_column_bounding_box_center)


In [ ]:
x_radius = y_radius= z_radius = 100_000

allen_center = alu.allen_column_bounding_box_center
(du.proofreading_neurons_table() 
 & f"(soma_x_nm < {allen_center[0] + x_radius}) AND (soma_x_nm > {allen_center[0] - x_radius})"
 & f"(soma_y_nm < {allen_center[1] + y_radius}) AND (soma_y_nm > {allen_center[1] - y_radius})"
 & f"(soma_z_nm < {allen_center[2] + z_radius}) AND (soma_z_nm > {allen_center[2] - z_radius})"
)

# Stacking Together Automatically (but uses the same parameter settings)

In [4]:
seg_1,split_1 = 864691134964458783,0
seg_2,split_2 = 864691134884754682,0
seg_3,split_3 = 864691134884952570,0

In [5]:
pv.plot_multiple_proofread_neuron(
segment_ids = [seg_1,seg_2],#,seg_3],
    split_indexes = [split_1,split_2],#,split_3],
    proofread_mesh_color = ["red","blue","purple"],
    synapse_plot_type = "valid_presyn_postsyn",
    plot_error_mesh = False,
    verbose = True
    
)   

NameError: name 'pv' is not defined

# Stack together multiple visualizations (can use different parameter settings)

In [ ]:
seg_1,split_1 = 864691134940132963,0
seg_2,split_2 = 864691134964569375,0

In [ ]:
pv.plot_proofread_neuron(seg_1,
                        split_1,
                         plot_error_mesh=True,
                        plot_synapses=True,
                         synapse_plot_type="compartment",
                         plot_error_synapses=False,
                         
                        print_spine_colors = False,
                        print_compartment_colors = False,
                         show_at_end = False,
                         append_figure = False)

pv.plot_proofread_neuron(seg_2,
                        split_2,
                         plot_error_mesh=False,
                        plot_synapses=True,
                         synapse_plot_type="valid_presyn_postsyn",
                         plot_error_synapses=False,
                         
                         show_at_end = True,
                         append_figure = True)
                         